In [5]:
import math
import numpy as np
import cv2 as cv
from numpy.linalg import inv

1. Copy all pixel values with colors within a specified range to a 3d array with x position y position and color value as white while leaving all other points black. This array is then written into an image.

In [ ]:
import numpy as np
import cv2 as cv

img = cv.imread('IMG_8617 (1).jpeg') # image to look at
img = cv.cvtColor(img, cv.COLOR_RGB2BGR)

im2write = np.zeros((1080,1920,3))

for x in range(1080):
   for y in range(1920):
           if((img[x][y][0] > 40 and img[x][y][0] < 110) and (img[x][y][1] > 40 and img[x][y][1] < 110) and (img[x][y][2] > 40 and img[x][y][2] < 110)):
                im2write[x][y][0] = 255
                im2write[x][y][0] = 255
                im2write[x][y][0] = 255
            

cv.imwrite('result5.png', im2write) # if you want filtered output image

2. filter excess noise using either an external program or a function that needs to be coded

3. Apply a hough line transform to the filtered image of the shadow, and remove excess lines

In [6]:
# Read image
image = cv.imread('bw2.png') # put a path to your image here
gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
 
# Use canny edge detection
edges = cv.Canny(gray,30,150,apertureSize=3)
 
# Obtain line end points
lines_list =[]
lines = cv.HoughLinesP(edges, 1, np.pi/180, threshold=150, minLineLength=30, maxLineGap=200)

cleans = np.empty(shape=[0,4], dtype=np.int32) # remove overlapping lines from HoughLines

for l in lines:
        
        # manually extending length of lines so that they all intersect, x can be altered 

        x = 50

        x0 = l[0][0]
        y0 = l[0][1]
        x1 = l[0][2]
        y1 = l[0][3]

        linSlope = (y1 - y0)/(x1 - x0)
        
        if x0 < x1:
             l[0][0] = x0 - x
             l[0][2] = x1 + x
        if x0 > x1: 
             l[0][0] = x0 + x
             l[0][2] = x0 - x

        if y0 < y1:
             l[0][1] = y0 - linSlope*(x)
             l[0][3] = y1 + linSlope*(x)
        if y0 > y1:
             l[0][1] = y0 - linSlope*(x)
             l[0][3] = y1 + linSlope*(x)

        # removing lines that are within 3.5 degrees of another and start and end within 30 pixels of one another, can be adjusted based on setup

        alfa = np.degrees(np.arctan2(l[0][2]-l[0][0], l[0][3]-l[0][1]))

        if len(cleans) == 0:
            cleans = np.append(cleans, [l[0]], axis=0)
            continue

        similar = False
        for c in cleans:
            beta = np.degrees(np.atan2(c[2]-c[0], c[3]-c[1]))
            other = False
            for d in cleans:
                 if(abs(l[0][0] - d[0]) <= 30 and abs(l[0][1] - d[1] <= 30) and abs(l[0][2] - d[2]) <= 30 and abs(l[0][3] - d[3] <= 30)):
                      other = True
            if abs(alfa-beta) <= 3.5 and other:
                similar = True
                other = False
                break

        if not similar:
            cleans = np.append(cleans, [l[0]], axis=0)
            

for line in [cleans]:
    for x1,y1,x2,y2 in line:
        cv.line(image,(x1,y1),(x2,y2),(0,0,255),2)


3. Construct line objects from line segment endpoints. Then corner objects are created if the line segments intersect.

In [7]:
def distanceForm(x1, y1, x2, y2):
     return '{0:.4f}'.format(math.sqrt((x1 - x2)**2 + (y1 - y2)**2))

class Line:
    
    def __init__(self, x0, y0, x1, y1):
        self.x0 = x0
        self.y0 = y0
        self.x1 = x1
        self.y1 = y1
        self.slope = float((y1 - y0)/(x1 - x0))
        self.theta = math.atan((y1-y0)/(x1-x0))
        self.r = math.dist([x0,y0],[x1,y1])
        self.intercept = y0 - self.slope*x0

    def __str__(self):
        return ("radius: " + str(self.r) + " theta: " + "{:.2f}".format(self.theta*180/math.pi) + " degrees")

    def getx0(self):
         return self.x0
    
    def gety0(self):
         return self.y0
    
    def getx1(self):
         return self.x1
    
    def gety1(self):
         return self.y1
    
    def getSlope(self):
        return self.slope
   
    def getTheta(self):
        return self.theta
    
    def getR(self):
        return self.r
    
    def getInter(self):
         return self.intercept


class intersectPoint:
        
        def __init__(self, l1, l2):
            
             self.l1 = l1
             self.l2 = l2
             self.line1 = Line(self.l1.getx0(),self.l1.gety0(),self.l1.getx1(),self.l1.gety1())
             self.line2 = Line(self.l2.getx0(),self.l2.gety0(),self.l2.getx1(),self.l2.gety1())
             self.poi = None 
             self.angle = None
             self.isCorner = False

             self.xends1 = [self.line1.getx0(), self.line1.getx1()]
             self.yends1 = [self.line1.gety0(), self.line1.gety1()]
             self.xends2 = [self.line2.getx0(), self.line2.getx1()]
             self.yends2 = [self.line2.gety0(), self.line2.gety1()]
             
             refA = np.array([[1., float(-1*self.line1.getSlope())], [1., float(-1*self.line2.getSlope())]])
             refS = np.array([[self.line1.getInter()],[self.line2.getInter()]])
             refInv = np.linalg.inv(refA)
             point = np.matmul(refInv, refS)
             self.theoPoi = point



             if(point[1] >= min(self.xends1) and point[1] <= max(self.xends1) and point[0] >= min(self.yends1) and point[0] <= max(self.yends1) and point[1] >= min(self.xends2) and point[1] <= max(self.xends2) and point[0] >= min(self.yends2) and point[0] <= max(self.yends2)):
            
                self.poi = point
                self.angle =  abs(self.l1.getTheta()-self.l2.getTheta())
                self.isCorner = True


        def __str__(self):
             return(self.line1, self.line2)
        
        def getLine1(self):
             return self.line1
        
        def getLine2(self):
             return self.line2
        
        def getPoi(self):
                return self.poi
        
        def getAngle(self):
                return self.angle
        
        def getAngleInDeg(self):
             return self.angle*180/math.pi
        
        def getTheoPoi(self):
             return self.theoPoi
        
        def getCorner(self):
             return self.isCorner


def lineListConstruct(pointsArr):     
     rows, columns = pointsArr.shape 
     lineList = []
     for i in range(rows):
          newLine = Line(pointsArr[i][0], pointsArr[i][1], pointsArr[i][2], pointsArr[i][3])
          lineList.append(newLine)
     return lineList

def intersectListConstruct(linesArr, k, storageArr = None, cornerArr = None):
     if storageArr is None:
          storageArr = [] 
     if cornerArr is None:
          cornerArr = []
     if (k == 1):
          for e in range(len(storageArr)):
               if storageArr[e].isCorner == True:
                    cornerArr.append(storageArr[e])
          return cornerArr
          
     else: 
          for i in range(len(linesArr) - 1):
               newPoi = intersectPoint(linesArr[0], linesArr[i + 1]) 
               storageArr.append(newPoi)
          k -= 1
          return intersectListConstruct(linesArr[1:], k, storageArr, cornerArr)


4. Convert endpoint data from hough lines image into a set of lines and corners.

In [26]:
lineList = []

for e in cleans:
    newList = [e[0], e[1], e[2], e[3]]
    lineList.append(newList)


lineArr = np.array(lineList)


testSet = lineListConstruct(lineArr)
testCornerSet = intersectListConstruct(testSet, len(testSet))

Feel free to test functionality here

In [27]:
def similarSet(c1, c2):
    if abs(c1.getAngleInDeg() - c2.getAngleInDeg()) <= 5:
        return True
    else:
        return False
    
for e in testCornerSet:
    print(similarSet(testCornerSet[0], e))

True
False
False
True
False
False


In [69]:
def similarCornerList(cornerSet, k=None, out=None):
    if out is None:
        out = []
    if k is None:
        k = len(cornerSet)

    if(k == 1):
        return out
    
    else:
        for e in cornerSet[1:]:
            if(similarSet(cornerSet[0], e) == True):
                out.append((cornerSet[0].getPoi(), e.getPoi(), distanceForm(cornerSet[0].getPoi()[0], cornerSet[0].getPoi()[1], e.getPoi()[0], e.getPoi()[1])))

        k -= 1
        return similarCornerList(cornerSet[1:], k, out)
    
    



In [70]:
test2 = similarCornerList(testCornerSet)
print(test2)

[(array([[655.61537794],
       [590.38086753]]), array([[699.40228766],
       [176.87661231]]), '415.8161'), (array([[183.73271052],
       [908.03846804]]), array([[152.14769834],
       [545.6420125 ]]), '363.7703'), (array([[ 985.36205096],
       [1079.51915488]]), array([[1075.89357275],
       [ 716.51412093]]), '374.1238')]


/var/folders/9r/k3ms4g2d4snb39037_z2v_rw0000gn/T/ipykernel_8177/4225617070.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return '{0:.4f}'.format(math.sqrt((x1 - x2)**2 + (y1 - y2)**2))
